# Contents

Pick up each light curve and guesstimate starting points for the MCMC fit

In [ ]:
# basics
import numpy as np
import pandas as pd


# matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

# constants and units
from astropy.constants import R_sun, R_jup
import astropy.units as u

# modulation model
from funcs.model import (full_model,
                         aflare,
                         aflare_decoupled,
                         calculate_specific_flare_flux,
                         calculate_angular_radius)


# data management
import os
CWD = "/".join(os.getcwd().split("/")[:-2])


# ignore some output
import warnings
warnings.simplefilter("ignore")


In [ ]:
# Read in target parameters
lcs = pd.read_csv(f"{CWD}/data/summary/lcs.csv")

# Choose target from table
target = lcs.iloc[3]

In [ ]:
# Pick up inclination and uncertainty (just for initial fits, we are using empirical priors later)
i_mu, i_sigma = target.i_mu, target.i_sigma

# Pick up quiescent luminosity
qlum = target.tlum * u.erg/u.s

# Pick up stellar radius
R = target.rad * R_sun

target

In [ ]:
# every timestamp uniquely maps to one light curve
tstamp ="10_12_2020_07_05"

In [ ]:
gplc = pd.read_csv(f"{CWD}/data/lcs/{tstamp}_{target.ID}.csv")
gplc.head()

# Play with parameters to get a first guess for the MCMC 

In [ ]:
# When fitting more than one flare, do one flare first, then the next etc.

# Flare peak time
phi_a =15.
# Active region latitude
theta_a = 1.4

# Flare amplitude
a = .5

# 2*FWHM of flare
fwhm1 = 4.96
fwhm2 = 4.96


phi0 =-2.61


# if there are two peaks
#---------------------------------------

# Flare peak time
# phi_a2 =17.8


# # Flare amplitude
# a2 = .05053675464244

# # 2*FWHM of flare
# fwhm12 = 1.1
# fwhm22 =1.4
#-----------------------------------------

Fth = calculate_specific_flare_flux("TESS", flaret=1e4)
radius = calculate_angular_radius(Fth, a, qlum, R,)

plt.figure(figsize=(10,7))
plt.plot(gplc.phi,gplc.flux)
phi= gplc.phi.values
median = gplc.median_.iloc[0]

for i in np.linspace(0,np.pi/2,10):
    plt.plot(phi, full_model(phi_a, theta_a, a, fwhm1, fwhm2, i, phi0=phi0,
               phi=phi, num_pts=50, qlum=qlum,
               Fth=Fth, R=R, median=median), c="r", alpha=.31,)
    
plt.plot(phi, full_model(phi_a, theta_a, a, fwhm1, fwhm2, i_mu, phi0=phi0,
           phi=phi, num_pts=50, qlum=qlum,
           Fth=Fth, R=R, median=median), c="k", alpha=1,);



# add this to the above if you are estimating two peaks

#  + full_model(phi_a2, theta_a, a2, fwhm12, fwhm22, i, phi0=phi0,
#                phi=phi, num_pts=50, qlum=qlum,
#                Fth=Fth, R=R, median=median)-median
    
#     + full_model(phi_a2, theta_a, a2, fwhm12, fwhm22, i_mu, phi0=phi0,
#                phi=phi, num_pts=50, qlum=qlum,
#                Fth=Fth, R=R, median=median)-median

# Write initializers to file

In [ ]:
# Different flare/active region configurations have different numbers of parameters
log_probs = {"log_probability": 7, # one flare, one active region
             "log_probability_2flares": 11, # two flares, one active region
            }

# Choose model:
log_prob = "log_probability_2flares"
nparam = log_probs[log_prob]


# Save starting points and parameters for MCMC
# one row per flare
# for two flare solutions, add suffix "a" and "b" to the ID below
suffix=""
with open(f"{CWD}/data/summary/inits_decoupled.csv", "a") as f:
    firstout = ("date,ID,phi_a,theta_a,a,fwhm1,fwhm2,Fth,phi0,"\
                "i_mu,i_sigma,radius_deg,R_Rsun,qlum_erg_s,"\
                "median,log_prob,nparam\n")
    out = (f"{tstamp},{target.ID}{suffix},{phi_a},{theta_a},{a},{fwhm1},{fwhm2},{Fth.value},"\
           f"{phi0},{i_mu},{i_sigma},{radius},{(R/R_sun).value},{qlum.value},"\
           f"{median},{log_prob},{nparam}\n")
   # f.write(firstout) #if you are starting a new table
    f.write(out)